# 00 - Environment Setup & Verification

**Author:** Tan Ming Kai (24PMR12003)  
**Date:** 2025-11-09  
**Purpose:** Verify GPU, dependencies, and environment configuration for CrossViT COVID-19 FYP

**Project:** Multi-Scale Vision Transformer (CrossViT) for COVID-19 Chest X-ray Classification  
**Hardware:** RTX 4060 8GB VRAM  
**Academic Year:** 2025/26

---

## Objectives
1. ✅ Verify GPU availability and CUDA compatibility
2. ✅ Check all required dependencies
3. ✅ Test CrossViT model loading from timm
4. ✅ Validate dataset paths
5. ✅ Set up reproducibility configuration
6. ✅ Test memory monitoring utilities

---

## 1. Reproducibility Setup & Critical Imports

**CRITICAL:** This section MUST run first in ALL notebooks to ensure reproducible results.

In [ ]:
"""
Environment Setup Notebook for CrossViT COVID-19 FYP
Author: Tan Ming Kai (24PMR12003)
Purpose: Verify all dependencies and hardware before starting data pipeline
"""

# ============================================================================
# 1. REPRODUCIBILITY SETUP (ALWAYS FIRST!)
# ============================================================================
import random
import numpy as np
import torch

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print("✅ Random seeds set to 42 for reproducibility")

# ============================================================================
# 2. STANDARD LIBRARY IMPORTS
# ============================================================================
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# 3. DATA SCIENCE LIBRARIES
# ============================================================================
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# ============================================================================
# 4. COMPUTER VISION LIBRARIES
# ============================================================================
import cv2
from PIL import Image
import timm  # For CrossViT model

# ============================================================================
# 5. PYTORCH & DEEP LEARNING
# ============================================================================
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

print("\n✅ All imports successful!")
print(f"PyTorch version: {torch.__version__}")
print(f"OpenCV version: {cv2.__version__}")
print(f"Timm version: {timm.__version__}")

✅ Random seeds set to 42 for reproducibility



✅ All imports successful!
PyTorch version: 2.6.0+cu124
OpenCV version: 4.12.0
Timm version: 1.0.22


## 2. GPU & CUDA Verification

**Expected Hardware:** NVIDIA RTX 4060 with 8GB VRAM

In [ ]:
# Check CUDA availability
print("=" * 70)
print("GPU & CUDA VERIFICATION")
print("=" * 70)

cuda_available = torch.cuda.is_available()
print(f"\n✓ CUDA Available: {cuda_available}")

if cuda_available:
    # Get GPU details
    gpu_name = torch.cuda.get_device_name(0)
    gpu_count = torch.cuda.device_count()
    cuda_version = torch.version.cuda
    
    print(f"✓ GPU Name: {gpu_name}")
    print(f"✓ GPU Count: {gpu_count}")
    print(f"✓ CUDA Version: {cuda_version}")
    
    # Get VRAM information
    gpu_properties = torch.cuda.get_device_properties(0)
    total_memory_gb = gpu_properties.total_memory / 1e9
    
    print(f"✓ Total VRAM: {total_memory_gb:.2f} GB")
    print(f"✓ GPU Compute Capability: {gpu_properties.major}.{gpu_properties.minor}")
    
    # Check current memory usage
    allocated_memory = torch.cuda.memory_allocated(0) / 1e9
    reserved_memory = torch.cuda.memory_reserved(0) / 1e9
    
    print(f"\n📊 Current Memory Status:")
    print(f"   - Allocated: {allocated_memory:.4f} GB")
    print(f"   - Reserved: {reserved_memory:.4f} GB")
    print(f"   - Free: {total_memory_gb - reserved_memory:.2f} GB")
    
    # Verify expected hardware
    if "4060" in gpu_name and 7.0 <= total_memory_gb <= 9.0:
        print("\n✅ CONFIRMED: RTX 4060 8GB detected - Ready for training!")
    else:
        print(f"\n⚠️  WARNING: Expected RTX 4060 8GB, but detected {gpu_name}")
        print(f"   This may affect batch size and training configuration.")
    
else:
    print("\n❌ ERROR: CUDA not available!")
    print("   Please check:")
    print("   1. NVIDIA GPU drivers installed")
    print("   2. CUDA toolkit installed")
    print("   3. PyTorch installed with CUDA support")
    print("\n   Install PyTorch with CUDA:")
    print("   pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118")

print("\n" + "=" * 70)

GPU & CUDA VERIFICATION

✓ CUDA Available: True
✓ GPU Name: NVIDIA GeForce RTX 4060 Laptop GPU
✓ GPU Count: 1
✓ CUDA Version: 12.4
✓ Total VRAM: 8.59 GB
✓ GPU Compute Capability: 8.9

📊 Current Memory Status:
   - Allocated: 0.0000 GB
   - Reserved: 0.0000 GB
   - Free: 8.59 GB

✅ CONFIRMED: RTX 4060 8GB detected - Ready for training!



## 3. CrossViT Model Loading Test

**Objective:** Verify that CrossViT-Tiny can be loaded and run on the GPU

In [ ]:
print("=" * 70)
print("CROSSVIT MODEL LOADING TEST")
print("=" * 70)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\n✓ Using device: {device}")

try:
    # Load CrossViT-Tiny model
    print("\n📥 Loading CrossViT-Tiny from timm library...")
    model = timm.create_model('crossvit_tiny_240', pretrained=True, num_classes=4)
    model = model.to(device)
    model.eval()
    
    print(f"✅ Model loaded successfully!")
    
    # Count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"\n📊 Model Statistics:")
    print(f"   - Total parameters: {total_params:,}")
    print(f"   - Trainable parameters: {trainable_params:,}")
    print(f"   - Model size: ~{total_params * 4 / 1e6:.2f} MB (FP32)")
    
    # Test forward pass
    print("\n🧪 Testing forward pass with dummy input...")
    dummy_input = torch.randn(1, 3, 240, 240).to(device)
    
    with torch.no_grad():
        output = model(dummy_input)
    
    print(f"✅ Forward pass successful!")
    print(f"   - Input shape: {dummy_input.shape}")
    print(f"   - Output shape: {output.shape}")
    print(f"   - Expected output shape: torch.Size([1, 4]) for 4 classes")
    
    if output.shape == torch.Size([1, 4]):
        print("\n✅ Model configuration CORRECT for COVID-19 4-class classification!")
    else:
        print(f"\n⚠️  WARNING: Output shape {output.shape} doesn't match expected [1, 4]")
    
    # Check memory usage after loading model
    if torch.cuda.is_available():
        model_memory = torch.cuda.memory_allocated(0) / 1e9
        print(f"\n📊 GPU Memory after model loading: {model_memory:.4f} GB")
        print(f"   - Estimated memory for batch_size=8: ~{model_memory * 8:.2f} GB")
        
        if model_memory * 8 < 7.5:  # Safe threshold for 8GB VRAM
            print("   ✅ Should fit comfortably in 8GB VRAM with batch_size=8")
        else:
            print("   ⚠️  May need smaller batch size or gradient accumulation")
    
    # Clean up
    del model, dummy_input, output
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    print("\n✅ CrossViT model test PASSED!")
    
except Exception as e:
    print(f"\n❌ ERROR loading CrossViT model: {e}")
    print("\nTroubleshooting:")
    print("1. Ensure timm is installed: pip install timm")
    print("2. Check internet connection (for pretrained weights)")
    print("3. Verify sufficient disk space for model download")

print("\n" + "=" * 70)

CROSSVIT MODEL LOADING TEST

✓ Using device: cuda

📥 Loading CrossViT-Tiny from timm library...


✅ Model loaded successfully!

📊 Model Statistics:
   - Total parameters: 6,725,960
   - Trainable parameters: 6,725,960
   - Model size: ~26.90 MB (FP32)

🧪 Testing forward pass with dummy input...


✅ Forward pass successful!
   - Input shape: torch.Size([1, 3, 240, 240])
   - Output shape: torch.Size([1, 4])
   - Expected output shape: torch.Size([1, 4]) for 4 classes

✅ Model configuration CORRECT for COVID-19 4-class classification!

📊 GPU Memory after model loading: 0.0362 GB
   - Estimated memory for batch_size=8: ~0.29 GB
   ✅ Should fit comfortably in 8GB VRAM with batch_size=8

✅ CrossViT model test PASSED!



## 4. Dataset Path Verification

**Expected:** COVID-19 Radiography Database with 21,165 images across 4 classes

In [ ]:
print("=" * 70)
print("DATASET PATH VERIFICATION")
print("=" * 70)

# Define expected dataset structure
DATA_ROOT = Path("../data/raw/COVID-19_Radiography_Dataset")

# Alternative paths to check
alternative_paths = [
    Path("../data/raw/COVID-19_Radiography_Dataset/COVID-19_Radiography_Dataset"),  # Nested structure
    Path("../data/raw/COVID-19_Radiography_Dataset"),
    Path("./data/raw/COVID-19_Radiography_Dataset"),
    Path("D:/Users/USER/Documents/Visual_Studio_Code/FYP_Code/data/raw/COVID-19_Radiography_Dataset"),
]

# Expected classes and approximate counts
EXPECTED_CLASSES = {
    "COVID": 3616,
    "Normal": 10192,
    "Lung_Opacity": 6012,
    "Viral Pneumonia": 1345
}

dataset_found = False
dataset_path = None

# Try to find the dataset
print("\n🔍 Searching for dataset...")
for path in alternative_paths:
    if path.exists():
        # Check if this path actually contains the class folders
        has_classes = any((path / class_name).exists() for class_name in EXPECTED_CLASSES.keys())
        if has_classes:
            print(f"✅ Found dataset at: {path.absolute()}")
            dataset_path = path
            dataset_found = True
            break
        else:
            print(f"   ⚠️  Path exists but no class folders: {path}")
    else:
        print(f"   ❌ Not found: {path}")

if dataset_found:
    print(f"\n📂 Dataset Location: {dataset_path.absolute()}")
    
    # Check for class folders
    print(f"\n📊 Class Distribution:")
    print("-" * 70)
    
    total_images = 0
    for class_name, expected_count in EXPECTED_CLASSES.items():
        class_path = dataset_path / class_name / "images"
        
        if class_path.exists():
            image_files = list(class_path.glob("*.png"))
            actual_count = len(image_files)
            total_images += actual_count
            
            # Check if count matches expected
            match_status = "✅" if abs(actual_count - expected_count) < 100 else "⚠️ "
            
            print(f"{match_status} {class_name:20s}: {actual_count:5d} images (expected ~{expected_count})")
            
            # Show sample image path
            if image_files:
                print(f"   Sample: {image_files[0].name}")
        else:
            print(f"❌ {class_name:20s}: Folder not found at {class_path}")
    
    print("-" * 70)
    print(f"   Total Images: {total_images} (expected ~21,165)")
    
    if 20000 <= total_images <= 22000:
        print("\n✅ Dataset verification PASSED!")
        print(f"✅ Dataset ready for loading in subsequent notebooks")
    else:
        print(f"\n⚠️  WARNING: Total image count {total_images} differs from expected ~21,165")
        print("   Please verify dataset integrity")
    
    # Test loading a single image
    print("\n🧪 Testing image loading...")
    try:
        # Find first image
        for class_name in EXPECTED_CLASSES.keys():
            class_path = dataset_path / class_name / "images"
            if class_path.exists():
                image_files = list(class_path.glob("*.png"))
                if image_files:
                    test_image_path = image_files[0]
                    
                    # Load with OpenCV
                    img_cv = cv2.imread(str(test_image_path), cv2.IMREAD_GRAYSCALE)
                    
                    # Load with PIL
                    img_pil = Image.open(test_image_path)
                    
                    print(f"✅ Successfully loaded: {test_image_path.name}")
                    print(f"   - OpenCV shape: {img_cv.shape}")
                    print(f"   - PIL size: {img_pil.size}")
                    print(f"   - PIL mode: {img_pil.mode}")
                    
                    break
    except Exception as e:
        print(f"❌ Error loading test image: {e}")
    
else:
    print("\n❌ ERROR: Dataset not found!")
    print("\n📋 Setup Instructions:")
    print("1. Download COVID-19 Radiography Database from Kaggle:")
    print("   https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database")
    print("\n2. Extract to: FYP_Code/data/raw/COVID-19_Radiography_Dataset/")
    print("\n3. Expected structure:")
    print("   data/raw/COVID-19_Radiography_Dataset/")
    print("   ├── COVID/images/")
    print("   ├── Normal/images/")
    print("   ├── Lung_Opacity/images/")
    print("   └── Viral Pneumonia/images/")

print("\n" + "=" * 70)

DATASET PATH VERIFICATION

🔍 Searching for dataset...
✅ Found dataset at: D:\Users\USER\Documents\GitHub\fyp-project\FYP_Code\notebooks\..\data\raw\COVID-19_Radiography_Dataset\COVID-19_Radiography_Dataset

📂 Dataset Location: D:\Users\USER\Documents\GitHub\fyp-project\FYP_Code\notebooks\..\data\raw\COVID-19_Radiography_Dataset\COVID-19_Radiography_Dataset

📊 Class Distribution:
----------------------------------------------------------------------
✅ COVID               :  3616 images (expected ~3616)
   Sample: COVID-1.png


✅ Normal              : 10192 images (expected ~10192)


   Sample: Normal-1.png
✅ Lung_Opacity        :  6012 images (expected ~6012)
   Sample: Lung_Opacity-1.png
✅ Viral Pneumonia     :  1345 images (expected ~1345)
   Sample: Viral Pneumonia-1.png
----------------------------------------------------------------------
   Total Images: 21165 (expected ~21,165)

✅ Dataset verification PASSED!
✅ Dataset ready for loading in subsequent notebooks

🧪 Testing image loading...


✅ Successfully loaded: COVID-1.png
   - OpenCV shape: (299, 299)
   - PIL size: (299, 299)
   - PIL mode: L



## 5. Project Configuration (Single Source of Truth)

All subsequent notebooks should import this configuration for consistency.

## 7. Environment Verification Summary

### Checklist Status

Run this notebook and verify all checks pass before proceeding to data loading:

- [ ] ✅ Random seeds set (seed=42)
- [ ] ✅ All dependencies imported successfully
- [ ] ✅ GPU detected (RTX 4060 8GB VRAM)
- [ ] ✅ CUDA available and compatible
- [ ] ✅ CrossViT-Tiny model loads correctly
- [ ] ✅ Forward pass successful (output shape: [1, 4])
- [ ] ✅ Dataset found at expected path
- [ ] ✅ All 4 class folders verified (~21,165 images)
- [ ] ✅ Sample images load successfully
- [ ] ✅ Project configuration created
- [ ] ✅ Memory monitoring utilities ready

---

### Next Steps

Once all checks pass above:

1. **Next Notebook:** `01_data_loading.ipynb`
   - Load all 21,165 images
   - Create train/val/test splits (80/10/10)
   - Save image paths to CSV for efficient loading
   - Verify class distribution

2. **Subsequent Pipeline:**
   ```
   02_data_cleaning.ipynb    → CLAHE enhancement, validation
   03_eda.ipynb              → Statistical analysis, visualizations
   04_data_augmentation.ipynb → Test augmentation strategies
   05_baseline_models.ipynb   → Train 5 baseline models
   06_crossvit_training.ipynb → Main model training
   07_results_analysis.ipynb  → Statistical tests, hypothesis validation
   08_ablation_studies.ipynb  → Test H2, H3, H4 hypotheses
   ```

---

### Troubleshooting

**If GPU not detected:**
- Check NVIDIA drivers: `nvidia-smi`
- Reinstall PyTorch with CUDA: `pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118`

**If CrossViT fails to load:**
- Check internet connection (downloads pretrained weights)
- Install timm: `pip install timm`
- Verify disk space for model cache (~100MB)

**If dataset not found:**
- Download from: https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database
- Extract to: `data/raw/COVID-19_Radiography_Dataset/`

---

**✅ Environment setup complete! Ready to begin FYP implementation.**

In [ ]:
def print_gpu_memory(prefix=""):
    """Print current GPU memory usage."""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated(0) / 1e9
        reserved = torch.cuda.memory_reserved(0) / 1e9
        total = torch.cuda.get_device_properties(0).total_memory / 1e9
        free = total - reserved
        
        print(f"{prefix}GPU Memory: Allocated={allocated:.3f}GB | Reserved={reserved:.3f}GB | Free={free:.3f}GB")
    else:
        print(f"{prefix}GPU not available")


def clear_memory():
    """Clear GPU cache and run garbage collection."""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    import gc
    gc.collect()


class GPUMemoryMonitor:
    """Context manager for monitoring GPU memory during operations."""
    
    def __init__(self, operation_name="Operation"):
        self.operation_name = operation_name
        self.start_memory = 0
        
    def __enter__(self):
        clear_memory()
        if torch.cuda.is_available():
            self.start_memory = torch.cuda.memory_allocated(0) / 1e9
        print(f"\n{'='*70}")
        print(f"Starting: {self.operation_name}")
        print_gpu_memory("  Before: ")
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if torch.cuda.is_available():
            end_memory = torch.cuda.memory_allocated(0) / 1e9
            delta = end_memory - self.start_memory
            print_gpu_memory("  After:  ")
            print(f"  Memory change: {delta:+.3f}GB")
        print(f"Completed: {self.operation_name}")
        print(f"{'='*70}\n")


# Test the utilities
print("=" * 70)
print("MEMORY MONITORING UTILITIES")
print("=" * 70)

print("\n✓ GPU memory utilities loaded:")
print("  - print_gpu_memory(prefix): Print current memory usage")
print("  - clear_memory(): Clear GPU cache")
print("  - GPUMemoryMonitor(name): Context manager for memory tracking")

print("\n🧪 Testing memory monitor...")
test_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
with GPUMemoryMonitor("Test Operation"):
    # Create a small tensor to demonstrate
    if torch.cuda.is_available():
        test_tensor = torch.randn(1000, 1000).to(test_device)
        del test_tensor

print("✅ Memory monitoring utilities ready!")
print("=" * 70)

MEMORY MONITORING UTILITIES

✓ GPU memory utilities loaded:
  - print_gpu_memory(prefix): Print current memory usage
  - clear_memory(): Clear GPU cache
  - GPUMemoryMonitor(name): Context manager for memory tracking

🧪 Testing memory monitor...

Starting: Test Operation
  Before: GPU Memory: Allocated=0.009GB | Reserved=0.021GB | Free=8.564GB


  After:  GPU Memory: Allocated=0.009GB | Reserved=0.021GB | Free=8.564GB
  Memory change: +0.000GB
Completed: Test Operation

✅ Memory monitoring utilities ready!


## 6. Memory Monitoring Utilities

Helper functions for tracking GPU memory during training.

In [ ]:
# ============================================================================
# PROJECT CONFIGURATION - Single Source of Truth
# ============================================================================

CONFIG = {
    # Reproducibility
    'seed': 42,
    
    # Hardware
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    
    # Data paths
    'data_root': Path('../data/raw/COVID-19_Radiography_Dataset'),
    'processed_data_dir': Path('../data/processed'),
    'output_dir': Path('../results'),
    'models_dir': Path('../models'),
    
    # Dataset specifications
    'image_size': 240,  # CrossViT-Tiny requirement
    'num_classes': 4,
    'class_names': ['COVID', 'Normal', 'Lung_Opacity', 'Viral Pneumonia'],
    'class_weights': [1.47, 0.52, 0.88, 3.95],  # For class imbalance
    
    # Data split
    'train_ratio': 0.8,
    'val_ratio': 0.1,
    'test_ratio': 0.1,
    
    # CLAHE parameters (for preprocessing)
    'clahe_clip_limit': 2.0,
    'clahe_tile_grid_size': (8, 8),
    
    # ImageNet normalization (required for pretrained models)
    'mean': [0.485, 0.456, 0.406],
    'std': [0.229, 0.224, 0.225],
    
    # Training hyperparameters (memory-safe for RTX 4060 8GB)
    'batch_size': 8,
    'gradient_accumulation_steps': 4,  # Effective batch size = 32
    'num_workers': 4,
    'pin_memory': True,
    'persistent_workers': True,
    
    # Optimizer settings
    'learning_rate': 5e-5,
    'weight_decay': 0.05,
    'max_epochs': 50,
    'early_stopping_patience': 15,
    
    # Mixed precision training
    'use_mixed_precision': True,
    
    # Logging
    'log_interval': 50,  # Print every 50 batches
    'save_interval': 5,  # Save checkpoint every 5 epochs
}

# Create necessary directories
CONFIG['processed_data_dir'].mkdir(parents=True, exist_ok=True)
CONFIG['output_dir'].mkdir(parents=True, exist_ok=True)
CONFIG['models_dir'].mkdir(parents=True, exist_ok=True)

print("=" * 70)
print("PROJECT CONFIGURATION")
print("=" * 70)
print(f"\n✓ Device: {CONFIG['device']}")
print(f"✓ Batch size: {CONFIG['batch_size']} (effective: {CONFIG['batch_size'] * CONFIG['gradient_accumulation_steps']})")
print(f"✓ Image size: {CONFIG['image_size']}×{CONFIG['image_size']}")
print(f"✓ Number of classes: {CONFIG['num_classes']}")
print(f"✓ Learning rate: {CONFIG['learning_rate']}")
print(f"✓ Max epochs: {CONFIG['max_epochs']}")
print(f"✓ Mixed precision: {CONFIG['use_mixed_precision']}")
print("\n✅ Configuration ready!")
print("=" * 70)

PROJECT CONFIGURATION

✓ Device: cuda
✓ Batch size: 8 (effective: 32)
✓ Image size: 240×240
✓ Number of classes: 4
✓ Learning rate: 5e-05
✓ Max epochs: 50
✓ Mixed precision: True

✅ Configuration ready!
